# Sentiment analysis / Reddit

### Extract posts from Reddit and I did sentiment analysis
### How the people looking to Israel 

In [ ]:
#import libraries

import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd
from datetime import datetime
import json

In [ ]:
# from Reddit put your details

client_id = "exQVPat6p_VJLp_YEYkx8A"
secret_key ="DUt6IpSWJ8otDXxLZUBju4W6N2UboA"

In [ ]:
#requests to APT Reddit
res = requests.post("https://www.reddit.com/api/v1/access_token", auth= auth, data= data, headers= headers)

In [ ]:
# create access token
TOKEN = res.json()["access_token"]

In [ ]:
data = {
    "grant_type" : "password",
    "username" : "****", #here put user name / Reddit Account
    "password" : "****"  #here put your password / Reddit Account
}

In [ ]:
headers= { 'user_agent': 'MyAPI/0.0.1'}

In [ ]:

# Your previous code to get the access token
# ...

# Specify the number of posts you want to retrieve per page (e.g., 100)
limit = 1000000   #
total_posts_to_retrieve = 1000000   #number of post
posts_per_request = 100

# Make requests to the Reddit API endpoint for hot posts in the "any words" subreddit with pagination
post_details = []
after = None

while len(post_details) < total_posts_to_retrieve:
    api_url = f'https://oauth.reddit.com/r/Israel/new.json?limit={limit}&after={after}' # your can replase any word /change Israel and you need hot or new you can change 
    headers = {'Authorization': f'Bearer {TOKEN}', 'User-Agent': 'MyAPI/0.0.1'}
    response = requests.get(api_url, headers=headers)

    # Check if the response status code is OK (200)
    if response.status_code == 200:
        try:
            # Parse the JSON response
            data = response.json()

            # Extract post details
            for post in data['data']['children']:
                title = post['data']['title']
                author = post['data']['author']
                created_utc = post['data']['created_utc']  # This is the timestamp of the post creation in UTC

                # Convert timestamp to a human-readable date and time
                created_datetime = datetime.utcfromtimestamp(created_utc).strftime('%Y-%m-%d')

                # Append post details to the list
                post_details.append({'Created': created_datetime, 'Title': title})

            # Get the ID of the last post in the current page for pagination
            after = data['data']['after']

        except Exception as e:
            print("Error processing JSON response:", str(e))
            break  # Break the loop if an error occurs
    else:
        print("Request failed with status code:", response.status_code)
        print("Response content:", response.content)
        break  # Break the loop if the request fails

# Create a DataFrame from the list of post details
df = pd.DataFrame(post_details)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification #model for Sentiment Analysis
from scipy.special import softmax

In [ ]:
# model Roberta for sentiment analysis

roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [ ]:
new= []

for i in df["Title"]:
    encoded_tweet = tokenizer(i, return_tensors='pt')
    output = model(**encoded_tweet)

    scores = output.logits[0].detach().numpy()
    probabilities = softmax(scores)

    max_index = probabilities.argmax()

    highest_sentiment = labels[max_index]
    highest_probability = probabilities[max_index]
    new.append(highest_sentiment)

In [ ]:
df["Sentiment Col"]= pd.Series(new)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
df.info()

plt.figure(figsize = [7, 7])
df["Sentiment Col"].value_counts().plot.pie(  autopct='%1.1f%%', shadow=True, startangle=90);
plt.title("Social Media Spotlight | Audience how looking to  Israel ")
plt.show()